In [ ]:
import os
import sys
import datetime

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# on my computer NX2 is not installed, I just import directly from the development directory
sys.path.append(os.path.abspath('../NX2/'))
import NX2
import NX2.polar
import NX2.math
sys.path.append(os.path.abspath('.'))
import utils

%matplotlib inline

ENTWURF = False

In [ ]:
datapath = '/home/moritz/Dropbox/NX2/'
datapath = '/melkor/d1/guenther/Dropbox/NX2/'
plotpath = os.path.join(datapath, '2018/')
geojsonpath = '/melkor/d1/guenther/projects/NX2/geojson/'

In [ ]:
from glob import glob

In [ ]:
# Move this into utils files once it's ready to be canned
filelist = glob(os.path.join(datapath, '2018', '*csv'))
filelist.sort()

In [ ]:
filelist

In [ ]:
data = [NX2.NX2Table(f, utils.date_from_filename(f)) for f in filelist]
# Compass data and all related columns are empty in first file (and thus deleted on read-in)
for n in ['HDC', 'DFT', 'SET']:
    data[0].add_column(n, np.nan)
# We don't have a rowfile with that info yet
for d in data:
    d.add_column('sailing', 0)

## Ein paar einfache plots nur um mal zu sehen, was denn da so vor sich geht

In [ ]:
for d in data:
    d.plot_course(scale=20, n=60)

In [ ]:
for i, d in enumerate(data):
    d.write_geojson(os.path.join(geojsonpath, '2018', os.path.basename(filelist[i]) + '.geojson'))

In [ ]:
plt.plot(data[2]['COG'], data[2]['HDC'])
plt.xlabel('COG')
plt.ylabel('HDC')

In [ ]:
d = data[2]
fit, ind = d.fit_BSP_corr()
fit.pprint()
plt.plot(d.SOG[~ind], d.BSP[~ind],'y.', label = 'data not used in fit')
plt.plot(d.SOG[ind], d.BSP[ind],'k.', label = 'data')
plt.plot(plt.xlim(), fit.beta * plt.xlim(), label = 'fit')
plt.xlabel('SOG')
plt.ylabel('BSP')
plt.legend(loc = 'upper left')

In [ ]:
import scipy.odr
self = data[2]
d = self

def line(B, x):
    ''' Linear function y = m*x + b '''
    return B[0] * x

con1 = (self.BSP > 0)  # moving
# SOG and BSP have different resonse times -> ignore gradients
smoothed = NX2.math.smooth_gauss(self.BSP, 3.)
con3 = (abs(np.diff(smoothed)) < 0.01)  # careful! n-1 elements!
myl = con3.tolist()
myl.append([True])
con3 = np.array(myl)
ind = con1 & con3
linear = scipy.odr.Model(line)
mydata = scipy.odr.RealData(self.SOG[ind], self.BSP[ind])
myodr = scipy.odr.ODR(mydata, linear, beta0=[1.])
fit = myodr.run()

fit.pprint()
plt.plot(d.SOG[~ind], d.BSP[~ind],'y.', label = 'data not used in fit')
plt.plot(d.SOG[ind], d.BSP[ind],'k.', label = 'data')
plt.plot(plt.xlim(), fit.beta * plt.xlim(), label = 'fit')
plt.xlabel('SOG')
plt.ylabel('BSP')
plt.legend(loc = 'upper left')

In [ ]:
fig = plt.figure(figsize=(12,6))
#fig.canvas.set_window_title('Bootsgeschwindigkeit')
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

sog = ax.plot(d.datetime(), d.AWA, label='AWA')
sog = ax.plot(d.datetime(), d.TWA, label='TWA')

sog = ax2.plot(d.datetime(), d.AWS, label='AWS')
sog = ax2.plot(d.datetime(), d.TWS, label='TWS')

for a in [ax, ax2]:
    a.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M', tz=None))
    xlab = a.get_xticklabels()
    for label in xlab: label.set_rotation(30)
    a.set_xlabel('Uhrzeit')
    a.legend()
ax2.set_ylabel('Geschwindigkeit [Knoten]')
ax.set_ylabel('Winkel')


In [ ]:
len(data)dd

In [ ]:
fig = plt.figure(figsize=(12, 12))

for i, d in enumerate(data):
    fit, ind = d.fit_BSP_corr()
    #fit.pprint()
    name = os.path.basename(filelist[i][:-7])
    print('{}: {}'.format(name, fit.beta))
    ax = fig.add_subplot(3, 4, i + 1)
    ax.plot(d.SOG[~ind], d.BSP[~ind],'y.', label = 'data not used in fit')
    ax.plot(d.SOG[ind], d.BSP[ind],'k.', label = 'data')
    ax.plot(plt.xlim(), fit.beta * plt.xlim(), label = 'fit')
    ax.set_xlabel('SOG')
    ax.set_ylabel('BSP')
    #ax.legend(loc = 'upper left')
    ax.set_title(name)

In [ ]:
np.sqrt(fit.cov_beta)

Vier Fahrten (drei am ersten Tag) haben keine schwarzen (nutzbaren) Punkte, weil der Kompass nicht so lief wie er sollte.

Da sehe ich erstmal das das trotz smoothing in einigen Fahrten BSPs > 40 auftreten. Vielleicht was das Grund Kontakt?

Die gefitteten Werte fur den Korrekturfaktor sind zwischen 0.8 und 0.9. Im Rahmen der Fehler sage ich jetzt einfach mal 0.85. Wenn ich mir die Zahlen anschaue, dann glaube ich nicht, dass die Unterschiede zwischen den Fahrten relevant sind. Ihr habt das Log also nicht zu doll angeknackst.

## Jetzt Daten noch mal einlesen, mit Korrekturfaktor

In [ ]:
for d in data:
    d.BSP /=0.85

In [ ]:
fig = plt.figure(figsize=(12, 12))
for i, d in enumerate(data):
    name = os.path.basename(filelist[i][:-7])
    ax = fig.add_subplot(4, 3, i + 1)
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M', tz=None))
    sog = ax.plot(d.datetime(), d.SOG, label='GPS')
    bsp = ax.plot(d.datetime(), d.BSP, label='Log - orginal')
    xlab = ax.get_xticklabels()
    for label in xlab: label.set_rotation(30)
    ax.set_xlabel('Uhrzeit')
    ax.set_ylabel('Geschwindigkeit [Knoten]')
    ax.set_title(name)
    #ax.legend()
fig.subplots_adjust(hspace=0.5)

In [ ]:
import mpld3

fig = data[9].plot_speeds()
mpld3.save_html(fig, '../test2018.html')
mpld3.display(fig)

In [ ]:
pwd

In [ ]:
for i, d in enumerate(data):
    name = os.path.basename(filelist[i][:-7])
    fig = plt.figure(figsize=(12, 4))
    ax = fig.add_subplot(111)
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M', tz=None))
    sog = ax.plot(d.datetime(), d.SOG, label='GPS')
    bsp = ax.plot(d.datetime(), d.BSP, label='Log - orginal')
    xlab = ax.get_xticklabels()
    for label in xlab: label.set_rotation(30)
    ax.set_xlabel('Uhrzeit')
    ax.set_ylabel('Geschwindigkeit [Knoten]')
    ax.set_title(name)
    ax.legend()
    mpld3.save_html(fig, os.path.join(datapath, '2018', 'figures', name + '.html'))
